In [0]:
# !pip install hyperopt

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from datetime import date, timedelta
from itertools import product
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
import xgboost as xgb 
from xgboost import plot_importance 
from sklearn import metrics 
from math import sqrt 
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from typing import List

In [0]:
from google.colab import drive 
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd "/content/gdrive/My Drive/Data_Mining_Cup/05 Code/it2_team1"

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Data_Mining_Cup/05 Code/it2_team1'
/content


In [0]:
data = pd.read_pickle("./processed_data/all_data.pkl")

FileNotFoundError: ignored

In [0]:
# data = data.loc[1:2]  # For testing

In [0]:
to_be_dropped = [c for c in data.columns if "-" in c or "category" in c or "manufacturer" in c or "brand" in c]
to_be_dropped.append("week_nr")
to_be_dropped.append("priceCluster")
data.drop(columns=to_be_dropped, inplace=True)

NameError: ignored

In [0]:
data

NameError: ignored

In [0]:
info = pd.read_pickle('./processed_data/info.pk')
#orders = pd.read_pickle('./processed_data/orders.pk')
#items = pd.read_pickle('./processed_data/items.pk')
#timedep = pd.read_pickle('./processed_data/timedep.pk')

FileNotFoundError: ignored

In [0]:
#all_data = pd.merge(timedep.reset_index(), items, how='left', on=['itemID'])

# the "avg" features are computed over the entire 180 days
# for predicting the last 2 weeks, the avg should only contain information besides the last 2 weeks 
# -> import updated data once ^ fixed

# Reset index

In [0]:
data = data.reset_index(drop=False)

# Scaling

TODO: Currently not working, cause of RAM

In [0]:
scaled_labels = ['daily_sold']

scaler_ob = StandardScaler(copy=False)
#data[scaled_labels]= scaler_ob.fit_transform(data[scaled_labels])


In [0]:
data = data[data.daily_sold > 0]

In [0]:
data

# Adding the label

After this step, data does not exist. Only X, y left

In [0]:
# This list describes what we want to predict.
labels: List[str] = ["daily_sold"]

##########################
#########################
# We need to put date also into y to make the split
# We also need itemID for the evaluation function to be able to get the price of the item

# _y = data[['date', 'itemID', *labels]].copy()
# # Encode itemID and daily_sold: `daily_sold << 14 + itemID`
# _y['y_true_encoded'] = _y.apply(lambda x: (int(x[2]) << 14) + x[1], axis=1)
# y = _y.drop(['itemID', *labels], axis=1)

y = data[['date', *labels]].copy()
X = data.drop(labels, axis=1).copy()
# del data

# Deletion of unused feature


In [0]:
# Here we can specify the list of unwanted features. The black sheep of features.
# But node the feature "date", we will delete lateron (if wanted). Anyways we need it 
# for splitting up the whole stuff
columns_to_delete: List[str] = ['itemID']


##########################
#########################
try:
  # do the deletion
  X = X.drop(columns_to_delete, axis=1)
except KeyError as e:
  print(f"I could not delete the columns {e} cause they were not there. \n But hey, they are gone, like intented :D") # I like this commend - from Min :p 

# Split into train/val/test

In [0]:
def split(X,y,training_end, validation_end, testing_end, split_feature='date', delete_split_feature=True):
  # Training
  print(f"CREATING TRAINING from [begin, {training_end}]")
  train_X = X[X[split_feature] <= training_end]
  train_y = y[y[split_feature] <= training_end]

  # Validation
  print(f"CREATING VALIDATION from ({training_end}, {validation_end}]")
  val_X = X[(X[split_feature] > training_end)]
  val_X = val_X[(val_X[split_feature] <= validation_end)]
  val_y = y[(y[split_feature]> training_end)]
  val_y = val_y[(val_y[split_feature] <= validation_end)]

   # Testing
  print(f"CREATING TESTING from ({validation_end}, {testing_end}]")
  test_X = X[X[split_feature] >= testing_end]
  test_y = y[y[split_feature] >= testing_end]

  if delete_split_feature:
    train_X = train_X.drop(split_feature, axis=1)
    train_y = train_y.drop(split_feature, axis=1)
    val_X = val_X.drop(split_feature, axis=1)
    val_y = val_y.drop(split_feature, axis=1)  # TYPO IN ORIGINAL NOTEBOOK (val_Y with capital Y)
    test_X = test_X.drop(split_feature, axis=1)
    test_y = test_y.drop(split_feature, axis=1)
  
  return (train_X, train_y), (val_X, val_y), (test_X, test_y)

train, val, test = split(X,y, '20180318', '20180411', '20180615')
del X
del y

In [0]:
x_train, y_train = train
x_test, y_test = test #train
x_val, y_val = val #train

In [0]:
x_train # here 15 days...

In [0]:
x_train

In [0]:
#train_features = all_data_training.drop(['daily_sold', 'date', 'itemID'], axis=1)
#train_labels = all_data_training['daily_sold']

# itemID ?? 

In [0]:
# here try both validation sets: 
# 1. fixed June 3 - June 16
# 2. shuffle randomly for train & val 

#x_train, x_val, y_train, y_val = tts(train_features, train_labels, shuffle = True, test_size = 0.2)

# XGBoost Regressor Model

### Custom Eval Metric 

The evaluation function is implemented as proposed by the competition. I implemented the objective function in a slightly different way:

Reasons:
- The original function is composed of two linear functions (one for too high prediction, one for too low).
- Therefore, the gradient is either 1 or -0.6 (depends on whether the prediction was too high or too low).
- The second gradient (hessian) is always 0.
- All this led to oscillating behaviour of the model (prediction too high -> too low -> too high -> too low ...)

What did I do to make it better?
- I calculated the first and second derivative based on the quadratic version of the original function (This doesn't change the optimum of the function).
- This made gradients "nicer" (they get scaled with the distance to the optimum) - if the prediction is very wrong, the gradient is big, and if the prediction is really good, the gradient is small (but I guess you know how gradients work :) )

(by Karin)

In [0]:
from sklearn.metrics import make_scorer

# def custom_loss(y_true_encoded, y_pred):
#     y_true = y_true_encoded >> 14
#     item_id = y_true_encoded & (2**14 - 1)
#     item_price = info.loc[item_id].simulationPrice
#     if y_true == y_pred:
#         revenue = item_price * y_pred 
#     elif y_true > y_pred:
#         revenue = item_price * y_pred 
#     else:
#         revenue = item_price * (y_true + 0.6 * (y_true - y_pred)) # here (y_true - y_pred) is negative 
#     return revenue 

def custom_loss(y_true, y_pred):
    y_pred = y_pred.get_label()
    item_price = 1
    revenue = np.zeros_like(y_true)

    mask1 = y_pred <= y_true
    revenue[mask1] = item_price * y_pred[mask1]

    mask2 = y_pred > y_true
    revenue[mask2] = item_price * (y_true + 0.6 * (y_true - y_pred))[mask2]  # here (y_true - y_pred) is negative 
    
    total_revenue = sum(revenue)
    return "negative-monetary-value", -total_revenue

# def custom_objective(y_true, y_pred):
#   # print(type(y_pred))

#   # Second derivative is 0
#   item_price = 1

#   grad = np.zeros_like(y_pred)
#   grad[y_pred < y_true] = -item_price
#   grad[y_pred > y_true] = 0.6 * item_price

#   # print(list(zip(y_pred, y_true, grad))[:20])

#   hess = np.zeros_like(y_pred)

#   return grad, hess

def custom_objective(y_true, y_pred):
  """
  Squared version of loss
  """
  item_price = 1
  mask1 = y_pred <= y_true
  mask2 = y_pred > y_true

  grad = np.zeros_like(y_pred)
  grad[mask1] = -(-2 * (item_price**2) * y_pred)[mask1]
  grad[mask2] = -(1.2 * item_price * (1.6 * item_price * y_true - 0.6 * item_price * y_pred))[mask2]

  hess = np.zeros_like(y_pred)
  hess[mask1] = -(-2 * (item_price**2))
  # hess[mask2] = -(-1.2 * 0.6 * item_price)
  hess[mask2] = -(-1.2 * 0.6 * (item_price**2))

  # print(list(zip(y_pred, y_true, grad, hess))[:3])
  
  return grad, hess

my_custom_score = make_scorer(custom_loss)  # , greater_is_better=True)

### Hyper parameters tuning 

In [0]:
# more parameters can be added here in the parameter space: 
# range can also be changed for more fine tuning 
para_space = {
            'n_estimators': hp.quniform('n_estimators', 10, 20, 1),
            'max_depth': hp.quniform("max_depth", 3, 10, 1),
            'min_child_weight': hp.quniform('min_child_weight', 1, 5, 1),
            'learning_rate':hp.quniform('learning_rate', 0.05, 0.2, 0.05),  
            'gamma': hp.quniform('gamma', 0.8, 1, 0.1),
            'subsample':hp.quniform('subsample', 0.7, 1, 0.1),
            'eval_metric': my_custom_score, # 'rmse',
            'tree_method':'gpu_hist',
            'objective': custom_objective,  # 'reg:squarederror', 
            'seed': 7
        }

In [0]:
def eval_scoring(para_space):
    print("Training with params:", para_space)

    xgb_model = xgb.XGBRegressor(
                          n_estimators = int(para_space['n_estimators']), 
                          max_depth = int(para_space['max_depth']),
                          min_child_weight=int(para_space['min_child_weight']),
                          learning_rate = para_space['learning_rate'],
                          gamma = para_space['gamma'],
                          subsample = para_space['subsample'],
                          feval = para_space['eval_metric'],  # Typo in original notebook; feval instead of eval_metric
                          tree_method = para_space['tree_method'],
                          objective = para_space['objective'],
                          seed = para_space['seed']
                         )
    
    
    eval_set = [train, val]
    
    xgb_model.fit(train[0], train[1], eval_set = eval_set, early_stopping_rounds = 10, verbose = False)

    train_pred = xgb_model.predict(train[0])
    train_rmse = sqrt(metrics.mean_squared_error(train[1], train_pred))

    val_pred = xgb_model.predict(val[0])
    val_rmse = sqrt(metrics.mean_squared_error(val[1], val_pred))
    # monetary_value = custom_loss(val[1], val_pred)
    
    print("Train RMSE: ", train_rmse)
    print("Val RMSE: ", val_rmse)
    # print("Monetary value: ", monetary_value)
    
    return {'loss':val_rmse, 'status': STATUS_OK }

In [0]:
def hyperpara_tuning(para_space):
    """
    Returns the best set of hyperparamters, 
    given a parameter space and a evaluation function 
    """
    
    best_model = fmin(fn=eval_scoring,
                      space=para_space,
                      algo=tpe.suggest,
                      max_evals= 20 # para can be changed here 
                     )
    
    return best_model 

In [0]:
# set runtime environment to GPU at: Runtime -> Change runtime type 

best_hyperparams = hyperpara_tuning(para_space)

In [0]:
print(best_hyperparams)

### final xgb model based on Grid Search

In [0]:
def best_xgb_model(x_train, y_train, x_val, y_val, best_hyperparams):
  xgb_model = xgb.XGBRegressor(n_estimators = int(best_hyperparams['n_estimators']), 
                               learning_rate = best_hyperparams['learning_rate'], 
                               gamma = best_hyperparams['gamma'],
                               max_depth = int(best_hyperparams['max_depth']), 
                               min_child_weight = int(best_hyperparams['min_child_weight']),
                               subsample = best_hyperparams['subsample'],
                               tree_metric = 'gpu_hist', 
                               objective = custom_objective,
                               feval = my_custom_score,
                               seed = 7)

  # xgb_model.fit(train[0], train[1], eval_set = eval_set, early_stopping_rounds = 10, verbose = False)
  xgb_model.fit(x_train, y_train, verbose = True, eval_metric = custom_loss, #'rmse',
                eval_set = [(x_train, y_train), (x_val, y_val)])
    
  return xgb_model

In [0]:
final_xgb_model = best_xgb_model(x_train, y_train, x_val, y_val, best_hyperparams)

### Model Results 

In [0]:
train_predictions = final_xgb_model.predict(x_train)
train_predictions = [round(i) for i in train_predictions]
print("Train RMSE : %.8g" % sqrt(metrics.mean_squared_error(y_train, train_predictions)))

val_predictions = final_xgb_model.predict(x_val)
val_predictions = [round(i) for i in val_predictions]
print("Val RMSE : %.8g" % sqrt(metrics.mean_squared_error(y_val, val_predictions)))

In [0]:
pred = final_xgb_model.predict(x_train)
comparison = list(zip(pred, y_train.values))
# for c in comparison:
#   # if c[1][0] > 0:
#   if c[0] > 0.24384:
#     print(c)
#     break
# c = comparison[-10:]
# print(c)
print(set(pred))

In [0]:
results = final_xgb_model.evals_result()
metric = 'negative-monetary-value'  # 'rmse'
epochs = len(results['validation_0'][metric])
x_axis = range(0, epochs)
fig, ax = plt.subplots()
# ax.plot(x_axis, results['validation_0'][metric], label='Train')
ax.plot(x_axis, results['validation_1'][metric], label='Val')
ax.legend()
plt.ylabel(metric)
plt.title('XGBoost Regression Error')
plt.show()

In [0]:
print(results['validation_0'][metric])
print(results['validation_1'][metric])

In [0]:
ax = plot_importance(final_xgb_model)
fig = ax.figure
fig.set_size_inches(9, 6)

# True Demand over the testing period (last 2 weeks) 

In [0]:
# TODO (Min): adapt code according to the newly imported pickle data 
# Create predicitons 

In [0]:
# ground_truth = orders_testing.groupby('itemID')['daily_sold'].sum().reset_index(name='demandTrue')

# all_items_id = list(range(1, 10464))
# items_demand_True = list(ground_truth['itemID'].unique())
# missing_ids = list(set(all_items_id)-set(items_demand_True))

# missing_ids_df = pd.DataFrame(columns=['itemID', 'demandTrue'])
# for id in missing_ids:
#     missing_ids_df = missing_ids_df.append({'itemID': id}, ignore_index=True)
# missing_ids_df.fillna(0, inplace = True)

# ground_truth = ground_truth.append(missing_ids_df)
# ground_truth = ground_truth.sort_values(['itemID']).reset_index(drop=True)

# ground_truth

# Very naive model...

In [0]:
# naive baseline just using "median"

# baseline_pred = df_daily_sales[['itemID','avg_daily_sold_over_training_days']].drop_duplicates().reset_index(drop=True)
# baseline_pred['avg_daily_sold_over_training_days'] = baseline_pred['avg_daily_sold_over_training_days']*14
# baseline_pred['avg_daily_sold_over_training_days'] = [round(i) for i in baseline_pred['avg_daily_sold_over_training_days']]
# print("Test RMSE : %.8g" % sqrt(metrics.mean_squared_error(ground_truth['demandTrue'], baseline_pred['avg_daily_sold_over_training_days'])))

### Test custom eval and obj function

In [0]:
def test_eval(y_true, y_pred):
  return "test_eval", y_true - y_pred

In [0]:
def test_obj(y_true, y_pred):
  return np.ones_like(y_true), np.ones_like(y_true)

In [0]:
test_xgb_model = xgb.XGBRegressor(
    feval = test_eval,
    objective = test_obj,
)

In [0]:
test_xgb_model.fit(x_train, y_train, verbose = True)